# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
#!pip install requests

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
City_Data_df = pd.read_csv("../output_data/cities.csv")
City_Data_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
Heatmap = City_Data_df[["Lat", "Lng"]]
Humidity = City_Data_df["Humidity"]
gmaps.configure(api_key = g_key)
fig = gmaps.figure()
Heat_Figure = gmaps.heatmap_layer(Heatmap, weights=Humidity, dissipating=False, max_intensity=300, point_radius=5)
fig.add_layer(Heat_Figure)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
Cities_Narrowed = City_Data_df.loc[(City_Data_df["Max Temp"] < 80) & (City_Data_df["Max Temp"] > 70)  \
                                   & (City_Data_df["Wind Speed"] < 10) \
                                   & (City_Data_df["Cloudiness"] == 0) ].dropna()
Cities_Narrowed

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = Cities_Narrowed
params = {"radius": 5000, "types": "lodging", "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row ["Lat"]
    lng = row ["Lng"]
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    print (f"Retrieving results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params = params).json()
    
    results = response['results']
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name']= results[0]['name']
        
    except (KeyError, IndexError):
        print("missing info...skipping")    

Retrieving results for Index 70: ponta do sol.
Closest hotel is Hotel Escarpas do Lago.
Retrieving results for Index 88: cidreira.
Closest hotel is Hotel Castelo.
Retrieving results for Index 138: morondava.
Closest hotel is La Case Bambou.
Retrieving results for Index 139: vaini.
Closest hotel is Dandeli Lake County.
Retrieving results for Index 176: nador.
Closest hotel is Hôtel La Giralda.
Retrieving results for Index 297: mogok.
Closest hotel is MOGOK HILL HOTEL.
Retrieving results for Index 444: birjand.
Closest hotel is تاکسی سرویس آرامش.


In [7]:
response

{'html_attributions': [],
 'next_page_token': 'ATtYBwJ56KPlNet0l-uXl4KeAVhzoIpt6HA5Q-n2A2Wh6lFkLCb1CYpLANBBrzlwR2E54Zxg8NAJOmnh1KiCd9Fek2GMLTfAER9vBvzLrBuItl-PT1TghDsF_-tSVtnHhss-R6IP1pejjcwD7Rrt_Y51pgnzP0j5_AcTYEdz2Y391_z62w5cXQZe7AgiINcG2gZZjfU5huxZ3yaO1P9KiCH3aIzStNsKwsvvTrbRHg_9jsURoBeSvf341oHnreVdWdysznBWDqLXMvLGB0X-JPVbKxW2FQ2IS2gkwA1EcPUXGwSzO5-V38Si4WVht5E1Zdei8QQrSBSfoAFJjjnJPppLxcFPYof9KMu3rSXBgxj5cHLBxjMQZYKqLVUzr0xbepoE-gyTELLSMmMVk6XMfxecTrU_-R1hxPuoD2smHvSGTgZYm9Owcnnancek',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 32.8625177, 'lng': 59.2037759},
    'viewport': {'northeast': {'lat': 32.86386753029149,
      'lng': 59.20508738029152},
     'southwest': {'lat': 32.86116956970849, 'lng': 59.2023894197085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'تاکسی سرویس آرامش',
   'place_id': 'ChIJnTu1F_tdGj8RpIt30JNB67Y',
   'plus_code': {'compound_code': 'V673+2G Birjand, South Kh

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
Heatmap = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
Hotel_Layer = gmaps.symbol_layer(Heatmap, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info)

# Display figure
Marker_Layer = gmaps.marker_layer(Heatmap, info_box_content=hotel_info)
fig.add_layer(Marker_Layer)
fig

Figure(layout=FigureLayout(height='420px'))